# `isochrones`

https://github.com/timothydmorton/isochrones

Stellar model grid access and model fitting

## What?

Use grids of stellar models to...

- Infer physical properties of exoplanet host stars
- Model potential transiting planet false positive scenarios
- Characterize visual binary systems
- Infer properties of star clusters

...conditioned on observations (broadband photometry, spectroscopy, asteroseismology, parallax, etc.)

## Why?

* Fundamental task in observational astronomy (what is this star, given these observations?)
* Reinvented probably hundreds of times with ad hoc algorithms
* Lack of clarity in the literature

### Kepler-22 (Borucki et al. 2012):
<img src=overview_images/kep22_describe.png>

<img src=overview_images/kep22-corner.png width=600>

## How?

Precomputed model grids (e.g. the MIST models) contain tables of observable and theoretical properties of stars as a function of various gridded parameters.  These grids can be organized in different ways, e.g.

- Isochrones: Grids in age and [Fe/H], each isochrone contains a range of masses*
- Evolutionary Tracks: Grids in mass and [Fe/H], each track has a range of ages*

**The MIST and Dartmouth grids define and use a quantity called EEP (equivalent evolutionary phase), which allows for regular gridding in the third dimension, and thus can serve as a proxy for mass (in isochrone grids) or age (in the evolutionary track grids).*

## Inference

Observed properties and uncertainties, e.g.: 

$$ \mathbf x = \{\bar J, \bar H, \bar K, \bar \pi\},~ \mathbf u = \{\bar J_{unc}, \bar H_{unc}, \bar K_{unc}, \bar \pi_{unc}\}$$

Parameters of model (single star):

$$\mathbf \theta = \{EEP, T, [Fe/H], d, AV \} $$

Likelihood:

$$p(\mathbf x~|~\mathbf \theta) \propto \prod ...$$ 

Computation of likelihood requires prediction of $x_i$ at arbitrary $\theta$ &rarr; **Requires interpolation**

## Implementation

In [1]:
from isochrones import get_ichrone

mist = get_ichrone('mist', bands='JHK') # Downloads & reorganizes appropriate data into ~/.isochrones (or $ISOCHRONES)
mist.df.head()

EEP  log10_isochrone_age_yr  initial_mass  \
feh  log10_isochrone_age_yr EEP                                              
-4.0 5.0                    35    35                     5.0      0.100000   
                            36    36                     5.0      0.102885   
                            37    37                     5.0      0.107147   
                            38    38                     5.0      0.111379   
                            39    39                     5.0      0.115581   

                                 star_mass  log_Teff     log_g     log_L  \
feh  log10_isochrone_age_yr EEP                                            
-4.0 5.0                    35    0.100000  3.617011  3.350571 -0.489734   
                            36    0.102885  3.618039  3.347798 -0.472691   
                            37    0.107147  3.619556  3.343658 -0.447471   
                            38    0.111379  3.621062  3.339612 -0.422498   
                            39    0.115581  3.622555  3.335660 -0.397776   

                                 [Fe/H]_init   [Fe/H]         H         J  \
feh  log10_isochrone_age_yr EEP                                             
-4.0 5.0                    35          -4.0 -3.90751  3.902331  4.446939   
                            36          -4.0 -3.90751  3.862969  4.407142   
                            37          -4.0 -3.90751  3.804645  4.348186   
                            38          -4.0 -3.90751  3.746857  4.289824   
                            39          -4.0 -3.90751  3.689636  4.232103   

                                        K   dm_deep  
feh  log10_isochrone_age_yr EEP                      
-4.0 5.0                    35   3.718756  0.000688  
                            36   3.680907  0.000681  
                            37   3.624838  0.000322  
                            38   3.569300 -0.000029  
                            39   3.514316 -0.000027

## Custom linear interpolation

In [2]:
mist.interp

In [3]:
mist.interp.grid.shape # filled-in regular grid (nan-padded)

(15, 107, 1711, 13)

In [4]:
mist.interp.index_columns  # feh, log(age), eep

(array([-4.  , -3.5 , -3.  , -2.5 , -2.  , -1.75, -1.5 , -1.25, -1.  ,
        -0.75, -0.5 , -0.25,  0.  ,  0.25,  0.5 ]),
 array([ 5.  ,  5.05,  5.1 ,  5.15,  5.2 ,  5.25,  5.3 ,  5.35,  5.4 ,
         5.45,  5.5 ,  5.55,  5.6 ,  5.65,  5.7 ,  5.75,  5.8 ,  5.85,
         5.9 ,  5.95,  6.  ,  6.05,  6.1 ,  6.15,  6.2 ,  6.25,  6.3 ,
         6.35,  6.4 ,  6.45,  6.5 ,  6.55,  6.6 ,  6.65,  6.7 ,  6.75,
         6.8 ,  6.85,  6.9 ,  6.95,  7.  ,  7.05,  7.1 ,  7.15,  7.2 ,
         7.25,  7.3 ,  7.35,  7.4 ,  7.45,  7.5 ,  7.55,  7.6 ,  7.65,
         7.7 ,  7.75,  7.8 ,  7.85,  7.9 ,  7.95,  8.  ,  8.05,  8.1 ,
         8.15,  8.2 ,  8.25,  8.3 ,  8.35,  8.4 ,  8.45,  8.5 ,  8.55,
         8.6 ,  8.65,  8.7 ,  8.75,  8.8 ,  8.85,  8.9 ,  8.95,  9.  ,
         9.05,  9.1 ,  9.15,  9.2 ,  9.25,  9.3 ,  9.35,  9.4 ,  9.45,
         9.5 ,  9.55,  9.6 ,  9.65,  9.7 ,  9.75,  9.8 ,  9.85,  9.9 ,
         9.95, 10.  , 10.05, 10.1 , 10.15, 10.2 , 10.25, 10.3 ]),
 array([0.000e+00, 1.000e+00, 

In [5]:
pars = [0.01, 9.54, 300.3]
mist.interp(pars, 'log_g')

ValueError: 'l' is not in list

### Comparison with SciPy

In [ ]:
from scipy.interpolate import RegularGridInterpolator

# Construct SciPy regular grid interpolator for logg
points = mist.interp.index_columns
values = mist.interp.grid[:, :, :, 5]  # logg is column 5
fn = RegularGridInterpolator(points, values)

In [ ]:
pars = [0.01, 9.54, 300.3]
fn(pars)

In [ ]:
%timeit fn(pars)

In [ ]:
mist.interp(pars, 'log_g')

In [ ]:
# convenience function for mist.interp(pars, 'log_g'), with different args
%timeit mist.logg(*pars[::-1]) 

Slightly slower than SciPy for vectorized calculations, however

In [ ]:
import numpy as np

N = 10000
%timeit mist.interp([np.ones(N)*0.01, np.ones(N)*9.54, np.ones(N)*300.3], 'log_g')

In [ ]:
%timeit fn(np.array([np.ones(N)*0.01, np.ones(N)*9.54, np.ones(N)*300.3]).T)

## `isochrones` in action

In [ ]:
from isochrones import StarModel
from isochrones import get_ichrone

mist = get_ichrone('mist', bands='JHK')

props = dict(Teff=(5642, 50.0), feh=(-0.27, 0.08), logg=(4.443, 0.028), 
             J=(10.523, 0.02), H=(10.211, 0.02), K=(10.152, 0.02))

# Single star model
mod = StarModel(mist, **props)
mod.print_ascii()

In [ ]:
mod.param_names

In [ ]:
p = [300, 9.5, 0.1, 300, 0.1]
mod.lnpost(p)

In [ ]:
%timeit mod.lnpost(p)

In [ ]:
mod.fit(basename='kep22')  # Defaults to using MultiNest if available

In [ ]:
mod.samples.describe()

Generate a corner plot with `mod.corner_physical()`:
<img src="overview_images/mist_corner_single_physical.png" width="600">

### Binary star model (unresolved)

In [ ]:
mod2 = StarModel(mist, N=2, **props)
mod2.print_ascii()

In [ ]:
mod2.param_names

<img src="overview_images/mist_corner_binary_physical.png" width=600>

&rarr; Can also do triple systems, resolved binaries, partially resolved binaries, etc.

In [ ]:
%%file demo_star/star.ini

Teff = 4135, 98.0
feh = -0.46, 0.16
logg = 4.711, 0.1

[twomass]
J = 13.513, 0.02
H = 12.845, 0.02
K = 12.693, 0.02

[NIRC2]
resolution = 0.1
separation_1 = 0.6
PA_1 = 100
K_1 = 3.66, 0.05
H_1 = 3.77, 0.03
J_1 = 3.74, 0.05
separation_2 = 1.2
PA_2 = 200
K_2 = 5.1, 0.1
H_2 = 5.2, 0.1
J_2 = 5.15, 0.1


In [ ]:
mod3 = StarModel.from_ini(mist, 'demo_star')
mod3.print_ascii()

In [ ]:
mod3.param_names

In [ ]:
mod4 = StarModel.from_ini(mist, 'demo_star', N=[2,1,1])
mod4.print_ascii()

In [ ]:
mod4.param_names

In [ ]:
mod5 = StarModel.from_ini(mist, 'demo_star', N=[2,1,1], index=[0,1,1])
mod5.print_ascii()

In [ ]:
mod5.param_names

## Fitting Clusters (new/in progress)

### Simulate observations of a cluster

In [ ]:
from isochrones import get_ichrone
from isochrones.cluster import SimulatedCluster

mist = get_ichrone('mist')

N = 50
cluster_pars = [8.84, -0.2, 500, 0.03, -3, 0.3, 0.3]
stars = SimulatedCluster(N, *cluster_pars, bands='gri', mass_range=(1, 2.5), phot_unc=0.01)
stars.df.head()

### Make a model

In [ ]:
from isochrones.cluster import StarClusterModel
model = StarClusterModel(mist, stars, eep_bounds=(200, 700), minq=0.5)
model.param_names

$$ p(M_1) \propto M_1^\alpha;~~ p(q = M_2/M_1) \propto q^\gamma$$

In [ ]:
isinstance(model, StarModel)

In [ ]:
model.lnpost(cluster_pars)

In [ ]:
%timeit model.lnpost(cluster_pars)

&rarr; Yikes! What's going on here?

In [ ]:
import holoviews as hv
hv.extension('bokeh')

In [ ]:
%%opts Points [width=400, height=400, tools=['hover']]
from isochrones.cluster import StarCatalog
import pandas as pd

df = pd.read_hdf('overview/small-test-cluster.h5')
test_stars = StarCatalog(df, bands='gri', props=['parallax'])
test_stars.hr

<img src="overview_images/cluster-test.png" width=600>

In [ ]:
%%opts Points [width=500, height=500, tools=['hover']]
data = hv.Points(test_stars.ds, kdims=['g-i', 'g_mag'], 
                 vdims=['is_binary', 'distance', 
                        'mass_pri', 'mass_sec', 
                        'eep_pri', 'eep_sec'], label='Simulated data').options(size=5)
model = mist.hr_isochrone('g', 'i', *cluster_pars[:4], mineep=300, maxeep=700, thin=2, label='Model isochrone').options(size=2)
data * model

## In the real world!

<img src="overview_images/hierarch-results.png" width=600>
Price-Whelan et al., in prep.